In [ ]:
from benchmark import Benchmark

In [ ]:
import hcipy
import numpy as np

def f(it, N):
    pupil_grid = hcipy.make_pupil_grid(N)

    wf = hcipy.Wavefront(pupil_grid.ones())

    apodizer = hcipy.Apodizer(pupil_grid.ones())

    for _ in it:
        apodizer(wf)

class NewApodizer(hcipy.OpticalElement):
    def __init__(self, apodization):
        self.apodization = apodization

    def forward(self, wf):
        return hcipy.Wavefront(wf.electric_field * self.apodization, wf.wavelength, wf.input_stokes_vector)

def g(it, N):
    pupil_grid = hcipy.make_pupil_grid(N)

    wf = hcipy.Wavefront(pupil_grid.ones())

    apodizer = NewApodizer(pupil_grid.ones())

    for _ in it:
        apodizer(wf)

class SimplerApodizer(hcipy.AgnosticOpticalElement):
    '''A thin apodizer.

    This apodizer can apodize both in phase and amplitude.

    Parameters
    ----------
    apodization : Field or scalar or function of wavelength
        The apodization that we want to apply to any input wavefront.
    '''
    def __init__(self, apodization):
        self._apodization = apodization

        hcipy.AgnosticOpticalElement.__init__(self, True, True)

    def make_instance(self, instance_data, input_grid, output_grid, wavelength):
        instance_data.apodization = self.evaluate_parameter(self.apodization, input_grid, output_grid, wavelength)

    @property
    def apodization(self):
        return self._apodization

    @apodization.setter
    def apodization(self, apodization):
        self._apodization = apodization

        self.clear_cache()

    def get_input_grid(self, output_grid, wavelength):
        return output_grid

    def get_output_grid(self, input_grid, wavelength):
        return input_grid

    @hcipy.make_agnostic_forward
    def forward(self, instance_data, wavefront):
        wf = hcipy.Wavefront(wavefront.electric_field * instance_data.apodization, wavefront.wavelength, wavefront.input_stokes_vector)

        return wf

    @hcipy.make_agnostic_backward
    def backward(self, instance_data, wavefront):
        wf = wavefront.copy()
        wf.electric_field *= instance_data.apodization.conj()

        return wf

def h(it, N):
    pupil_grid = hcipy.make_pupil_grid(N)

    wf = hcipy.Wavefront(pupil_grid.ones())

    apodizer = SimplerApodizer(pupil_grid.ones())

    for _ in it:
        apodizer(wf)

In [ ]:
bm = Benchmark('Apodization', 'N', [2**i for i in range(2, 14)], t_min=0.1)

bm.run(f, 'hcipy.Apodizer')
bm.run(g, 'NewApodizer')
bm.run(h, 'SimplerApodizer')

bm.plot()